In [1]:
!apt install -y influxdb-client

E: Could not open lock file /var/lib/dpkg/lock - open (13: Permission denied)
E: Unable to lock the administration directory (/var/lib/dpkg/), are you root?


In [2]:
!influx --version

InfluxDB shell 0.10.0


In [4]:
!influx -host 172.17.0.2 -database 'telegraf' -execute 'show measurements'

name: measurements
------------------
name
cpu
disk
diskio
kernel
mem
processes
swap
system



In [6]:
!influx -host 172.17.0.2 -database 'telegraf' -execute 'select * from cpu limit 5' -format 'json' -pretty

{
    "results": [
        {
            "series": [
                {
                    "name": "cpu",
                    "columns": [
                        "time",
                        "cpu",
                        "host",
                        "usage_guest",
                        "usage_guest_nice",
                        "usage_idle",
                        "usage_iowait",
                        "usage_irq",
                        "usage_nice",
                        "usage_softirq",
                        "usage_steal",
                        "usage_system",
                        "usage_user"
                    ],
                    "values": [
                        [
                            1533057550000000000,
                            "cpu-total",
                            "influx-cluster-1",
                            0,
                            0,
                            98.34876157116969,
                            0.025018764073031

In [7]:
!influx -host 172.17.0.2 -database 'telegraf' -execute 'select * from cpu limit 5' 

name: cpu
---------
time			cpu		host			usage_guest	usage_guest_nice	usage_idle		usage_iowait		usage_irq	usage_nice	usage_softirq		usage_steal	usage_system		usage_user
1533057550000000000	cpu-total	influx-cluster-1	0		0			98.34876157116969	0.02501876407303131	0		0		0.15011258443833006	0		0.5504128096072577	0.9256942707027273
1533057550000000000	cpu0		influx-cluster-1	0		0			97.9999999998865	0.09999999999976353	0		0		0.09999999999987011	0		0.6999999999989132	1.0999999999996726
1533057550000000000	cpu1		influx-cluster-1	0		0			98.19999999999709	0			0		0		0.29999999999999805	0		0.5000000000001137	0.9999999999990905
1533057550000000000	cpu2		influx-cluster-1	0		0			98.30169830167914	0			0		0		0.19980019980023192	0		0.499500499500693	0.9990009990025216
1533057550000000000	cpu3		influx-cluster-1	0		0			98.89669007020493	0			0		0		0			0		0.5015045135403923	0.6018054162481287



In [9]:
!influx -host 172.17.0.4 -database 'telegraf' -execute 'select * from cpu limit 5' 

ERR: error authorizing query: create admin user first or disable authentication
error authorizing query: create admin user first or disable authentication


In [13]:
!influx -host 172.17.0.2 -execute 'show databases' 

name: databases
---------------
name
_internal
telegraf



In [16]:
#via haproxy
!influx -host 172.17.0.5 -database 'telegraf' -execute 'select * from cpu limit 5' 

name: cpu
---------
time			cpu		host			usage_guest	usage_guest_nice	usage_idle		usage_iowait		usage_irq	usage_nice	usage_softirq		usage_steal	usage_system		usage_user
1533057550000000000	cpu-total	influx-cluster-1	0		0			98.34876157116969	0.02501876407303131	0		0		0.15011258443833006	0		0.5504128096072577	0.9256942707027273
1533057550000000000	cpu0		influx-cluster-1	0		0			97.9999999998865	0.09999999999976353	0		0		0.09999999999987011	0		0.6999999999989132	1.0999999999996726
1533057550000000000	cpu1		influx-cluster-1	0		0			98.19999999999709	0			0		0		0.29999999999999805	0		0.5000000000001137	0.9999999999990905
1533057550000000000	cpu2		influx-cluster-1	0		0			98.30169830167914	0			0		0		0.19980019980023192	0		0.499500499500693	0.9990009990025216
1533057550000000000	cpu3		influx-cluster-1	0		0			98.89669007020493	0			0		0		0			0		0.5015045135403923	0.6018054162481287



In [34]:
!influx -host 172.17.0.5 -username admin -password admin -execute 'create database iot' 

In [38]:
!influx -host 172.17.0.5 -database iot -username admin -password admin -execute "INSERT treasures,captain_id=pirate_king value=2"

In [40]:
!influx -host 172.17.0.5 -database iot -username admin -password admin -execute 'select * from treasures' 

name: treasures
---------------
time			captain_id	value
1533086889824245989	pirate_king	2



In [41]:
!influx -host 172.17.0.2 -database iot -username admin -password admin -execute 'select * from treasures' 

name: treasures
---------------
time			captain_id	value
1533086889824245989	pirate_king	2



In [42]:
!influx -host 172.17.0.3 -database iot -username admin -password admin -execute 'select * from treasures' 

name: treasures
---------------
time			captain_id	value
1533086889824245989	pirate_king	2



In [43]:
!influx -host 172.17.0.4 -database iot -username admin -password admin -execute 'select * from treasures' 

name: treasures
---------------
time			captain_id	value
1533086889824245989	pirate_king	2



In [44]:
!influx -host 172.17.0.5 -database telegraf -username admin -password admin -execute 'show measurements' 

name: measurements
------------------
name
cpu
disk
diskio
kernel
mem
processes
swap
system



In [45]:
!influx -host 172.17.0.3 -database telegraf -username admin -password admin -execute 'show measurements' 

name: measurements
------------------
name
cpu
disk
diskio
kernel
mem
processes
swap
system



In [46]:
!influx -host 172.17.0.4 -database telegraf -username admin -password admin -execute 'show measurements' 

name: measurements
------------------
name
cpu
disk
diskio
kernel
mem
processes
swap
system



## high availability

In [47]:
#shutdown node-1
!docker ps

CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS               NAMES
8cd4a384151f        haproxy             "/docker-entrypoint.…"   6 hours ago         Up 6 hours                              my-running-haproxy
9e04dcafd7a6        ubuntu:16.04        "sleep 9999999999999…"   27 hours ago        Up 27 hours                             influx-cluster-3
4a5974247f70        ubuntu:16.04        "sleep 9999999999999…"   27 hours ago        Up 27 hours                             influx-cluster-2
232ae55d8ab4        ubuntu:16.04        "sleep 9999999999999…"   27 hours ago        Up 27 hours                             influx-cluster-1


In [48]:
!docker stop 232

232


In [49]:
!influx -host 172.17.0.5 -database telegraf -username admin -password admin -execute 'show measurements' 

name: measurements
------------------
name
cpu
disk
diskio
kernel
mem
processes
swap
system



In [50]:
!influx -host 172.17.0.2 -database telegraf -username admin -password admin -execute 'show measurements' 

Failed to connect to http://172.17.0.2:8086
Please check your connection settings and ensure 'influxd' is running.


In [51]:
#shutdown node-2

In [52]:
!docker stop 4a5

4a5


In [53]:
!docker ps

CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS               NAMES
57dc8116bf4e        haproxy             "/docker-entrypoint.…"   37 seconds ago      Up 36 seconds                           my-running-haproxy-2
8cd4a384151f        haproxy             "/docker-entrypoint.…"   6 hours ago         Up 6 hours                              my-running-haproxy
9e04dcafd7a6        ubuntu:16.04        "sleep 9999999999999…"   27 hours ago        Up 27 hours                             influx-cluster-3
4a5974247f70        ubuntu:16.04        "sleep 9999999999999…"   27 hours ago        Up About a minute                       influx-cluster-2
232ae55d8ab4        ubuntu:16.04        "sleep 9999999999999…"   27 hours ago        Up About a minute                       influx-cluster-1


In [54]:
#use 2nd haproxy

In [55]:
!influx -host 172.17.0.6 -database telegraf -username admin -password admin -execute 'show measurements' 

name: measurements
------------------
name
cpu
disk
diskio
kernel
mem
processes
swap
system



## scalability

In [1]:
# add new node to the cluster

In [2]:
!docker run -d --name influx-cluster-4 --hostname influx-cluster-4 ubuntu:16.04 sleep 9999999999999999999

d51989491f3a8db845dea8527c06b08404f50b230019f2b687284d7f04dedbcf


In [3]:
!docker inspect d51

[
    {
        "Id": "d51989491f3a8db845dea8527c06b08404f50b230019f2b687284d7f04dedbcf",
        "Created": "2018-08-01T02:14:43.319021757Z",
        "Path": "sleep",
        "Args": [
            "9999999999999999999"
        ],
        "State": {
            "Status": "running",
            "Running": true,
            "Paused": false,
            "Restarting": false,
            "OOMKilled": false,
            "Dead": false,
            "Pid": 12205,
            "ExitCode": 0,
            "Error": "",
            "StartedAt": "2018-08-01T02:14:43.679403847Z",
            "FinishedAt": "0001-01-01T00:00:00Z"
        },
        "Image": "sha256:7aa3602ab41ea3384904197455e66f6435cb0261bd62a06db1d8e76cb8960c42",
        "ResolvConfPath": "/var/lib/docker/containers/d51989491f3a8db845dea8527c06b08404f50b230019f2b687284d7f04dedbcf/resolv.conf",
        "HostnamePath": "/var/lib/docker/containers/d51989491f3a8db845dea8527c06b08404f50b230019f2b687284d7f04dedbcf/hostname",
        "HostsPat

In [ ]:
# Add manually from docker exec for server-4
echo -e "172.17.0.2\tinflux-cluster-1" >> /etc/hosts
echo -e "172.17.0.3\tinflux-cluster-2" >> /etc/hosts
echo -e "172.17.0.4\tinflux-cluster-3" >> /etc/hosts

In [4]:
!docker exec -it influx-cluster-4 cat /etc/hosts

127.0.0.1	localhost
::1	localhost ip6-localhost ip6-loopback
fe00::0	ip6-localnet
ff00::0	ip6-mcastprefix
ff02::1	ip6-allnodes
ff02::2	ip6-allrouters
172.17.0.7	influx-cluster-4
172.17.0.2	influx-cluster-1
172.17.0.3	influx-cluster-2
172.17.0.4	influx-cluster-3


In [5]:
#install required package

In [ ]:
wget https://dl.influxdata.com/enterprise/releases/influxdb-meta_1.5.4-c1.5.4_amd64.deb
sudo dpkg -i influxdb-meta_1.5.4-c1.5.4_amd64.deb

In /etc/influxdb/influxdb-meta.conf:

uncomment and set hostname to the full hostname of the meta node set license-key in the [enterprise] section to the license key you received on InfluxPortal OR license-path in the [enterprise] section to the local path to the JSON license file you received from InfluxData.

In [ ]:
nohup influxd-meta --config /etc/influxdb/influxdb-meta.conf &

In [ ]:
wget https://dl.influxdata.com/enterprise/releases/influxdb-data_1.5.4-c1.5.4_amd64.deb

sudo dpkg -i influxdb-data_1.5.4-c1.5.4_amd64.deb

First, in /etc/influxdb/influxdb.conf, uncomment:

hostname at the top of the file and set it to the full hostname of the data node

auth-enabled in the [http] section and set it to true

shared-secret in the [http] section and set it to a long pass phrase that will be used to sign tokens for intra-cluster communication. This value needs to be consistent across all data nodes.

Note: When you enable authentication, InfluxDB only executes HTTP requests that are sent with valid credentials. See the authentication section for more information.

Second, in /etc/influxdb/influxdb.conf, set:

license-key in the [enterprise] section to the license key you received on InfluxPortal OR license-path in the [enterprise] section to the local path to the JSON license file you received from InfluxData.

In [ ]:
nohup influxd --config /etc/influxdb/influxdb.conf &

In [23]:
# join the cluster
!docker exec -it influx-cluster-4 influxd-ctl join -v influx-cluster-1:8091

Joining meta node at influx-cluster-1:8091
Searching for meta node on influx-cluster-4:8091...
Searching for data node on influx-cluster-4:8088...

Successfully joined cluster

  * Added meta node 8 at influx-cluster-4:8091
  * Added data node 7 at influx-cluster-4:8088


In [24]:
!docker exec -it influx-cluster-1 influxd-ctl show

Data Nodes
ID	TCP Address		Version
2	influx-cluster-1:8088	1.5.4-c1.5.4
4	influx-cluster-2:8088	
6	influx-cluster-3:8088	1.5.4-c1.5.4
7	influx-cluster-4:8088	1.5.4-c1.5.4

Meta Nodes
TCP Address		Version
influx-cluster-1:8091	1.5.4-c1.5.4
influx-cluster-2:8091	
influx-cluster-3:8091	1.5.4-c1.5.4
influx-cluster-4:8091	1.5.4-c1.5.4
